# Talkset 비윤리 강도 Regression 모델 학습 (KoBERT 기반)

Talkset 데이터셋의 `intensity` (비윤리 강도) 값을 예측하는 KoBERT 기반 Regression 모델을 학습합니다.

## Talkset 데이터 구조

Talkset 데이터셋에는 다음 필드가 있습니다:
- `is_immoral`: boolean (윤리적 문제 여부)
- `intensity`: float (비윤리 강도 점수, 0.0 ~ 약 2.0)
- `intensity_sum`: int (강도 합계)
- `types`: list (비윤리 유형, 예: "HATE", "CENSURE" 등)
- `text`: str (발화 텍스트)

**학습 목표**: 발화 텍스트를 입력받아 비윤리 강도(intensity)를 예측하는 Regression 모델 학습


## 1. 환경 설정 및 라이브러리 설치

Colab에서 실행 시 첫 번째 셀에서 라이브러리를 설치합니다.


In [ ]:
# 필요한 라이브러리 설치
!pip install transformers torch numpy pandas scikit-learn tqdm

# GPU 확인
import torch
print(f'CUDA 사용 가능: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')


## 2. 라이브러리 Import 및 설정

Python 라이브러리를 import하고 기본 설정을 합니다.


In [ ]:
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 장치: {device}')
print('라이브러리 import 완료')


## 3. Talkset 데이터 구조 확인

Talkset 데이터셋에는 다음 필드가 있습니다:
- `is_immoral`: boolean (윤리적 문제 여부)
- `intensity`: float (비윤리 강도 점수, 0.0 ~ 약 2.0) - **이것이 예측 대상**
- `intensity_sum`: int (강도 합계)
- `types`: list (비윤리 유형, 예: "HATE", "CENSURE" 등)
- `text`: str (발화 텍스트) - **이것이 입력**

**비윤리 강도 학습 체계:**
- Talkset 데이터셋에 `intensity` 필드가 이미 구현되어 있음
- 다수의 평가자(voter)가 각 문장의 비윤리 강도를 평가
- `intensity`는 평균 강도 값 (0.0 ~ 약 2.0)
- `intensity_sum`은 모든 평가자의 강도 합계
- `is_immoral=True`인 문장은 `intensity > 0`을 가짐

In [ ]:
# 데이터 파일 확인 (Colab에서는 파일 업로드 필요)
# !wget https://example.com/talksets-train-6.json  # 또는 파일 직접 업로드
# from google.colab import files
# uploaded = files.upload()

# 샘플 데이터 확인
data_file = 'talksets-train-6.json'  # 실제 경로로 수정

try:
    with open(data_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f'총 대화 수: {len(data)}')
    
    # 첫 번째 비윤리 문장 찾기
    for talk in data[:10]:
        for sentence in talk.get('sentences', []):
            if sentence.get('is_immoral', False):
                print(f'\n샘플 비윤리 문장:')
                print(f'  텍스트: {sentence.get("text", "")}')
                print(f'  is_immoral: {sentence.get("is_immoral")}')
                print(f'  intensity: {sentence.get("intensity")}')
                print(f'  intensity_sum: {sentence.get("intensity_sum")}')
                print(f'  types: {sentence.get("types", [])}')
                break
        else:
            continue
        break
    
    # 통계 계산
    all_sentences = [s for talk in data for s in talk.get('sentences', [])]
    immoral_sentences = [s for s in all_sentences if s.get('is_immoral', False)]
    
    print(f'\n전체 문장 수: {len(all_sentences):,}')
    print(f'비윤리 문장 수: {len(immoral_sentences):,}')
    
    if immoral_sentences:
        intensities = [s.get('intensity', 0.0) for s in immoral_sentences]
        print(f'비윤리 문장 Intensity 통계:')
        print(f'  평균: {np.mean(intensities):.3f}')
        print(f'  최소: {np.min(intensities):.3f}')
        print(f'  최대: {np.max(intensities):.3f}')
        print(f'  표준편차: {np.std(intensities):.3f}')
        
except FileNotFoundError:
    print(f'파일을 찾을 수 없습니다: {data_file}')
    print('Colab에서는 파일을 업로드하거나 경로를 수정하세요.')


## 4. 데이터 로드 및 전처리

Talkset JSON 파일을 로드하고, 학습/검증/테스트 데이터로 분할합니다.


In [ ]:
def load_talkset_data(file_path: str, filter_immoral_only: bool = False) -> Tuple[List[str], List[float]]:
    """
    Talkset 데이터 로드
    
    Args:
        file_path: talkset JSON 파일 경로
        filter_immoral_only: True이면 is_immoral=True인 문장만 로드
    
    Returns:
        (texts, intensities) 튜플
    """
    print(f'데이터 로드 중: {file_path}')
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    texts = []
    intensities = []
    
    for talk in tqdm(data, desc='대화 처리 중'):
        for sentence in talk.get('sentences', []):
            text = sentence.get('text', '').strip()
            is_immoral = sentence.get('is_immoral', False)
            intensity = sentence.get('intensity', 0.0)
            
            # 빈 텍스트 제외
            if not text:
                continue
            
            # 필터링 조건
            if filter_immoral_only and not is_immoral:
                continue
            
            texts.append(text)
            intensities.append(intensity)
    
    return texts, intensities

# 데이터 로드
all_texts, all_intensities = load_talkset_data(
    'talksets-train-6.json',
    filter_immoral_only=False  # 모든 문장 사용 (정상 발화는 intensity=0)
)

print(f'\n총 문장 수: {len(all_texts):,}')
print(f'  - intensity > 0: {sum(1 for i in all_intensities if i > 0):,}개')
print(f'  - intensity = 0: {sum(1 for i in all_intensities if i == 0):,}개')
print(f'\nIntensity 통계:')
print(f'  평균: {np.mean(all_intensities):.3f}')
print(f'  최소: {np.min(all_intensities):.3f}')
print(f'  최대: {np.max(all_intensities):.3f}')
print(f'  표준편차: {np.std(all_intensities):.3f}')


In [ ]:
# 데이터 분할 (학습/검증/테스트)
# 1차 분할: 학습+검증 / 테스트
train_val_texts, test_texts, train_val_intensities, test_intensities = train_test_split(
    all_texts,
    all_intensities,
    test_size=0.1,
    random_state=42,
    shuffle=True
)

# 2차 분할: 학습 / 검증
train_texts, val_texts, train_intensities, val_intensities = train_test_split(
    train_val_texts,
    train_val_intensities,
    test_size=0.1,
    random_state=42,
    shuffle=True
)

print(f'학습 데이터: {len(train_texts):,}개')
print(f'검증 데이터: {len(val_texts):,}개')
print(f'테스트 데이터: {len(test_texts):,}개')
print(f'\n학습 데이터 Intensity 통계:')
print(f'  평균: {np.mean(train_intensities):.3f}')
print(f'  최소: {np.min(train_intensities):.3f}')
print(f'  최대: {np.max(train_intensities):.3f}')


## 5. Dataset 클래스 정의

PyTorch Dataset 클래스를 정의하여 talkset 데이터를 모델 학습에 사용할 수 있는 형태로 변환합니다.


In [ ]:
class ImmoralityIntensityDataset(Dataset):
    """비윤리 강도 Regression을 위한 Dataset"""
    
    def __init__(
        self,
        texts: List[str],
        intensities: List[float],
        tokenizer,
        max_length: int = 128
    ):
        self.texts = texts
        self.intensities = intensities
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self) -> int:
        return len(self.texts)
    
    def __getitem__(self, idx: int) -> Dict:
        text = str(self.texts[idx])
        intensity = float(self.intensities[idx])
        
        # 토크나이징
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(intensity, dtype=torch.float32)
        }

print('Dataset 클래스 정의 완료')


## 6. KoBERT 모델 및 토크나이저 로드

KoBERT 모델을 로드하고 Regression 태스크에 맞게 설정합니다.

**KoBERT Regression 모델 구조:**
1. KoBERT Encoder: 입력 텍스트를 벡터로 변환
2. Regression Head: 출력 레이어에서 `num_labels=1`로 설정하여 하나의 실수 값(intensity) 예측
3. Loss Function: MSE (Mean Squared Error) 사용


In [ ]:
# KoBERT 모델명
MODEL_NAME = 'monologg/kobert'  # 또는 'skt/kobert-base-v1'

print(f'KoBERT 모델 로드 중: {MODEL_NAME}')

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print('토크나이저 로드 완료')

# Regression 모델 로드 (num_labels=1: 하나의 실수 값 예측)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1,  # Regression: 하나의 실수 값 출력
    problem_type="regression"  # Regression 태스크 명시
)

# GPU로 이동
model = model.to(device)
print(f'모델 로드 완료 (장치: {device})')

# 모델 파라미터 수 확인
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'총 파라미터 수: {total_params:,}')
print(f'학습 가능한 파라미터 수: {trainable_params:,}')


## 7. Dataset 객체 생성


In [ ]:
# Dataset 객체 생성
MAX_LENGTH = 128  # 최대 토큰 길이

train_dataset = ImmoralityIntensityDataset(
    train_texts,
    train_intensities,
    tokenizer,
    max_length=MAX_LENGTH
)

val_dataset = ImmoralityIntensityDataset(
    val_texts,
    val_intensities,
    tokenizer,
    max_length=MAX_LENGTH
)

test_dataset = ImmoralityIntensityDataset(
    test_texts,
    test_intensities,
    tokenizer,
    max_length=MAX_LENGTH
)

print(f'학습 Dataset 크기: {len(train_dataset)}')
print(f'검증 Dataset 크기: {len(val_dataset)}')
print(f'테스트 Dataset 크기: {len(test_dataset)}')


## 8. 평가 지표 함수 정의

Regression 모델 평가를 위한 지표 함수를 정의합니다:
- MSE (Mean Squared Error)
- MAE (Mean Absolute Error)
- R² Score
- RMSE (Root Mean Squared Error)


In [ ]:
def compute_metrics(eval_pred):
    """평가 지표 계산"""
    predictions, labels = eval_pred
    
    # predictions는 (batch_size, 1) 형태의 배열
    # labels는 (batch_size,) 형태의 배열
    predictions = predictions.flatten()
    labels = labels.flatten()
    
    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    r2 = r2_score(labels, predictions)
    
    # RMSE 계산
    rmse = np.sqrt(mse)
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }

print('평가 지표 함수 정의 완료')


## 9. 학습 설정 (TrainingArguments)

**KoBERT 학습 로직 설명:**
1. **사전 학습된 가중치 사용**: `monologg/kobert`는 한국어 코퍼스로 사전 학습된 BERT 모델
2. **Fine-tuning**: Regression 태스크에 맞게 출력 레이어를 수정하고 전체 모델을 미세 조정
3. **Learning Rate**: 2e-5 (BERT 모델의 일반적인 학습률, 작게 설정하여 사전 학습된 가중치를 과도하게 변경하지 않음)
4. **Batch Size**: GPU 메모리에 맞게 조정 (16 또는 32)
5. **Loss Function**: MSE Loss (Regression 태스크의 표준 손실 함수)


In [ ]:
# 학습 설정
OUTPUT_DIR = './kobert_immorality_intensity_regression'

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,  # 학습 에포크 수
    per_device_train_batch_size=16,  # 학습 배치 크기
    per_device_eval_batch_size=32,   # 평가 배치 크기
    learning_rate=2e-5,  # 학습률 (BERT 모델의 일반적인 학습률)
    weight_decay=0.01,   # L2 정규화
    warmup_steps=500,    # Warmup 스텝 수
    
    # 평가 및 저장 설정
    evaluation_strategy='epoch',  # 각 에포크마다 평가
    save_strategy='epoch',       # 각 에포크마다 저장
    save_total_limit=3,          # 최대 모델 파일 수 (이전 모델 자동 삭제)
    load_best_model_at_end=True, # 학습 종료 시 최적 모델 로드
    metric_for_best_model='mse', # 최적 모델 선택 기준 (MSE 낮을수록 좋음)
    greater_is_better=False,     # MSE는 낮을수록 좋으므로 False
    
    # 로깅 설정
    logging_dir=f'{OUTPUT_DIR}/logs',
    logging_steps=100,  # 100 스텝마다 로그 출력
    
    # 기타 설정
    seed=42,
    fp16=True,  # Mixed precision training (GPU 메모리 절약 및 속도 향상)
    dataloader_num_workers=2,  # 데이터 로딩 병렬 처리
    report_to='none'  # TensorBoard 등 외부 로깅 비활성화 (필요시 변경)
)

print('학습 설정 완료')


## 10. Trainer 객체 생성 및 학습

**학습 과정 설명:**
1. **Forward Pass**: 입력 텍스트를 KoBERT로 인코딩하여 벡터 표현 추출
2. **Regression Head**: 마지막 레이어에서 하나의 실수 값(intensity) 예측
3. **Loss Calculation**: 예측값과 실제 intensity 값의 MSE 계산
4. **Backward Pass**: 역전파를 통해 모델 파라미터 업데이트
5. **Evaluation**: 검증 데이터로 모델 성능 평가 (MSE, MAE, R²)


In [ ]:
# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 조기 종료
)

print('Trainer 객체 생성 완료')
print('학습 시작...')

# 모델 학습
train_history = trainer.train()

print('\n학습 완료!')
print(f'학습 시간: {train_history.metrics["train_runtime"]:.2f}초')


## 12. 테스트 데이터 평가 (최종 평가)

최종 테스트 데이터로 모델의 일반화 성능을 평가합니다.


In [ ]:
# 테스트 데이터로 최종 평가
print('=== 테스트 데이터 평가 (최종 평가) ===\n')
test_results = trainer.evaluate(test_dataset)

print('📊 [최종 테스트 결과]')
print(f'  MSE: {test_results["eval_mse"]:.4f}')
print(f'  RMSE: {test_results["eval_rmse"]:.4f}')
print(f'  MAE: {test_results["eval_mae"]:.4f}')
print(f'  R² Score: {test_results["eval_r2"]:.4f}')


## 11. 테스트 데이터 평가


In [ ]:
# 테스트 데이터로 최종 평가
print('테스트 데이터 평가 중...')
test_results = trainer.evaluate(test_dataset)

print('\n=== 테스트 결과 ===')
print(f'MSE: {test_results["eval_mse"]:.4f}')
print(f'RMSE: {test_results["eval_rmse"]:.4f}')
print(f'MAE: {test_results["eval_mae"]:.4f}')
print(f'R² Score: {test_results["eval_r2"]:.4f}')


## 12. 예측 예시

학습된 모델로 새로운 텍스트의 비윤리 강도를 예측합니다.


In [ ]:
# 학습된 모델로 예측
def predict_intensity(model, tokenizer, text: str, device) -> float:
    """단일 텍스트의 비윤리 강도 예측"""
    model.eval()
    
    # 토크나이징
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    # GPU로 이동
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # 예측
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predicted_intensity = outputs.logits.item()
    
    return max(0.0, predicted_intensity)  # 음수 방지

# 예측 예시
sample_texts = [
    "안녕하세요, 문의드리고 싶은 것이 있습니다.",
    "지금 당장 해결해달라니깐!",
    "원래 틀딱들은 눈치가 없어서 ㅋㅋㅋ",
    "해결 못하면 고소하겠습니다."
]

print('=== 예측 예시 ===')
for text in sample_texts:
    predicted = predict_intensity(model, tokenizer, text, device)
    print(f'\n발화: {text}')
    print(f'예측된 비윤리 강도: {predicted:.3f}')


## 13. 모델 저장

학습된 모델과 토크나이저를 저장합니다.


In [ ]:
# 최종 모델 저장
final_model_path = f'{OUTPUT_DIR}/final_model'
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f'✓ 모델이 저장되었습니다: {final_model_path}')

# 평가 결과 저장
results_summary = {
    'validation': {
        'mse': float(val_results['eval_mse']),
        'rmse': float(val_results['eval_rmse']),
        'mae': float(val_results['eval_mae']),
        'r2': float(val_results['eval_r2'])
    },
    'test': {
        'mse': float(test_results['eval_mse']),
        'rmse': float(test_results['eval_rmse']),
        'mae': float(test_results['eval_mae']),
        'r2': float(test_results['eval_r2'])
    },
    'training_info': {
        'train_samples': len(train_texts),
        'val_samples': len(val_texts),
        'test_samples': len(test_texts),
        'model_name': MODEL_NAME,
        'max_length': MAX_LENGTH,
        'train_files': TRAIN_DATA_FILES,
        'val_file': VAL_DATA_FILE
    }
}

# 결과 요약 저장
results_file = f'{OUTPUT_DIR}/evaluation_results.json'
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

print(f'✓ 평가 결과가 저장되었습니다: {results_file}')

print('\n' + '='*60)
print('📦 저장된 파일 목록:')
print(f'  1. 모델 파일: {final_model_path}/')
print(f'     - pytorch_model.bin')
print(f'     - config.json')
print(f'     - tokenizer 관련 파일들')
print(f'  2. 평가 결과: {results_file}')
print('='*60)
print('\n📖 모델 적용 방법:')
print('   프로젝트의 MODEL_APPLICATION_GUIDE.md 파일을 참조하세요.')
print('   또는 Colab에서 다음 명령으로 파일 다운로드:')
print('   from google.colab import files')
print('   !zip -r model.zip kobert_immorality_intensity_regression/final_model')
print('   files.download("model.zip")')


## 15. 결과 파일 다운로드 (Colab)

Colab에서 학습한 모델을 로컬로 다운로드하는 방법입니다.

**주의**: Colab 세션이 종료되면 파일이 삭제되므로, 반드시 다운로드하거나 Google Drive에 저장하세요.


In [ ]:
# 방법 1: 압축 파일로 다운로드
from google.colab import files
import shutil

# 모델 폴더 압축
zip_path = 'kobert_immorality_model.zip'
shutil.make_archive('kobert_immorality_model', 'zip', final_model_path)

print(f'✓ 압축 파일 생성: {zip_path}')
print('다운로드 중...')
files.download(zip_path)

# 평가 결과도 다운로드
files.download(results_file)

print('\n✓ 모든 파일 다운로드 완료!')


### Google Drive에 저장하는 방법 (선택사항)

Google Drive에 저장하면 나중에 다시 사용할 수 있습니다.


In [ ]:
# 방법 2: Google Drive에 저장 (선택사항)
# 주석을 해제하여 사용하세요

# from google.colab import drive
# drive.mount('/content/drive')

# # Drive에 복사
# import shutil
# drive_model_path = '/content/drive/MyDrive/kobert_immorality_intensity_model'
# shutil.copytree(final_model_path, drive_model_path, dirs_exist_ok=True)
# shutil.copy(results_file, '/content/drive/MyDrive/evaluation_results.json')

# print(f'✓ 모델이 Google Drive에 저장되었습니다: {drive_model_path}')


## 학습 요약

### KoBERT Regression 모델 학습 로직:

1. **모델 구조**:
   - KoBERT Encoder (12 layers, 768 hidden size)
   - Regression Head (Linear layer: 768 → 1)

2. **학습 방법**:
   - **사전 학습된 가중치 활용**: `monologg/kobert`는 한국어로 사전 학습된 BERT 모델
   - **Fine-tuning**: Regression 태스크에 맞게 출력 레이어만 추가하고 전체 모델 미세 조정
   - **학습률**: 2e-5 (작은 값으로 사전 학습된 가중치를 서서히 조정)
   - **Loss**: MSE (Mean Squared Error)

3. **데이터 흐름**:
   - 입력: 발화 텍스트 → KoBERT 토크나이저 → 토큰 IDs
   - KoBERT 인코딩: 토큰 IDs → 벡터 표현 (768차원)
   - Regression: 벡터 표현 → Linear Layer → intensity 값 (실수)
   - Loss: 예측 intensity vs 실제 intensity의 MSE

4. **최적화**:
   - AdamW Optimizer (BERT 모델의 표준 옵티마이저)
   - Learning Rate Warmup (초기 학습률을 점진적으로 증가)
   - Early Stopping (검증 성능이 개선되지 않으면 조기 종료)

5. **평가 지표**:
   - MSE: 평균 제곱 오차 (낮을수록 좋음)
   - RMSE: 평균 제곱근 오차 (실제 intensity 단위)
   - MAE: 평균 절대 오차 (실제 intensity 단위)
   - R²: 결정계수 (1에 가까울수록 좋음)
